In [1]:
print('hello, world!')

hello, world!


In [2]:
import os
from data import SQLData
from datasets import Dataset
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [3]:
HUGGING_FACE_WRITE_TOKEN = os.getenv("HUGGING_FACE_WRITE_TOKEN")

hf_spider_dev = 'alagaesia/spider_dev'
hf_spider_dev_preds = 'alagaesia/spider_dev_preds'

In [4]:
sqp = SQLData.from_dataset(hf_spider_dev)

In [5]:
sqp.data[hf_spider_dev]['train'][0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'question': 'How many singers do we have?',
 'create_w_keys': 'CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") ); ',
 'create_wo_keys': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity

In [6]:
def request_creation(
    strings: list[str],
    create_statement: str,
    question: str,
    create_statement_index: int,
    question_index: int,
) -> str: 
    
    modified_strings = strings.copy()
    
    modified_strings.insert(create_statement_index, create_statement)
    modified_strings.insert(question_index, question)

    return ' '.join(modified_strings)

def dataset_request_creation(
    dataset: Dataset,
    strings: list[str],
    request_column: str,
    create_statement_column: str,
    question_column: str,
    create_statement_index: int,
    question_index: int,
) -> str: 
    
    create_statement = dataset[create_statement_column]
    question = dataset[question_column]

    request_string = request_creation(
        strings,
        create_statement,
        question,
        create_statement_index,
        question_index,
    )

    return {request_column: request_string}


In [7]:
request_column = 'zero_shot_request'
create_statement_column = 'create_w_keys'
question_column = 'question'

strings = ['Given the following SQL schema: ', 'Provide a SQL query that answers the following question: ', 'Only respond with the SQL query.']

strings = [
    '### Complete sqlite SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query.\n',
    '### Sqlite SQL tables, with their properties:\n',
    '#\n',
    '# ', # create statement goes here (index 4)
    '\n', 
    '#\n',
    '### ', # question goes here (index 8) - must account for prior insertion
    '\n', 
    'SELECT',
]

create_statement_index = 4
question_index = 8

In [8]:
sqp.data[hf_spider_dev]['train'] = sqp.data[hf_spider_dev]['train'].map(dataset_request_creation, fn_kwargs = {'strings': strings, 'request_column': request_column, 'create_statement_column': create_statement_column, 'question_column': question_column, 'create_statement_index': create_statement_index, 'question_index': question_index}, load_from_cache_file=False)

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [9]:
sqp.data[hf_spider_dev]['train'][0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'question': 'How many singers do we have?',
 'create_w_keys': 'CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") ); ',
 'create_wo_keys': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity

In [10]:
print(sqp.data[hf_spider_dev]['train'][0]['zero_shot_request'])

### Complete sqlite SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query.
 ### Sqlite SQL tables, with their properties:
 #
 #  CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") );  
 #
 ###  How many singers do we 

In [11]:
sqp.hub_upload(
    repo_id=hf_spider_dev,
    token=HUGGING_FACE_WRITE_TOKEN,
    dataset=sqp.data[hf_spider_dev],
)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]